Milestone 4 - Model Training

Task 2: Perform feature selection

This task will focus on the second step of the workflow mentioned in the previous task. Feature selection is a process that allows you to select a subset of the features in your dataset. This is useful if you have a lot of features and you don't want to use all of them. It's great if you have a model that overfits the data, and you want to reduce the number of features. Perform feature selection, so you can use the most important features to train your model. You can use LASSO regression for this task, but it's not recommended since it doesn't work very well with multiclass classification. Instead, you can simply take a look at the weights of your features and see which ones are important. Remove those that have low weights and check again the performance. Before you do that, you should check the performance of the model without feature selection on both training and testing sets, and observe if, by removing some features, the metrics on both sets get closer. Don't worry if you underfit right now, you will improve your model later.

In [ ]:
import re
import pandas as pd
import os
import numpy as np
from csv import reader
import plotly.express as px
import missingno as msno
import pickle

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

pd.options.mode.chained_assignment = None